This notebook visualizes the participant compliance of `mBrain21` participants throughout the study.


In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../..')

from code_utils.path_conf import mbrain_metadata_path, processed_mbrain_path, figure_dir
from code_utils.utils.interaction_analysis import get_wearable_session_df
from code_utils.mbrain.event_parsing import EventDumpParser
from code_utils.mbrain.visualization import construct_headache_event_hovertext
from functional import seq
from datetime import datetime

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# whether the figure is displayed in the notebook via a static image or interactive plot
USE_PNG = True

## Interaction rate


utility functions:


In [2]:
def interaction_rate(df) -> pd.Series:
    return 100 * np.clip(
        (df["nb_confirmed"] + df["nb_manual_inserted"])
        / np.clip(df["nb_timeline"], a_min=1, a_max=None),
        a_min=0,
        a_max=1,
    )


def get_stress_event_interaction_df(df_eventdump) -> pd.DataFrame:
    # analyze the timeline events
    # 1. Stress events
    df_stress_events = EventDumpParser.parse_stress_events(df_eventdump)
    if not len(df_stress_events):
        return pd.DataFrame()
    df_stress_events = df_stress_events.set_index("time", drop=True)

    for c in ["prediction", "confirmed", "deprecated"]:
        if c not in df_stress_events.columns:
            df_stress_events[c] = np.NaN

    df_stress_metadata_day = []
    t_start = df_stress_events.index.min()
    t_end = df_stress_events.index.max()
    for day in pd.date_range(start=t_start.date(), end=t_end.date(), freq="D"):
        # rate = (manual events + confirmed predicted events) / total_predicted events
        # with a daily mask

        day_mask = df_stress_events.index.date == day.date()

        if not sum(day_mask):
            df_stress_metadata_day.append([day.date(), 0, 0, 0, 0])
            continue

        df_stress_day = df_stress_events[day_mask]

        nb_user_confirmations = len(
            df_stress_day[
                df_stress_day.prediction
                & df_stress_day.confirmed
                & (df_stress_day.deprecated != True)
            ]
        )

        nb_manual_events = 0
        if "deprecation_contexts" in df_stress_day.columns:
            # deprecated stress events "BY USER" + context -> set stress level = 0
            deprecated_events = len(
                df_stress_day[
                    (df_stress_day.deprecated_by == "user")
                    & (df_stress_day.deprecation_contexts.notna())
                ]
            )
            nb_manual_events += deprecated_events

        nb_user_deleted_events = 0
        if "deprecated_by" in df_stress_day.columns:
            nb_user_deleted_events = int(sum(df_stress_day.deprecated_by == "user"))

        nb_manual_events += len(
            df_stress_day[
                # prediction and deprecated can thus be Nan
                (df_stress_day.prediction != True)
                & (df_stress_day.deprecated != False)
            ]
        )
        nb_timeline_events = len(
            df_stress_day[df_stress_day.prediction & (df_stress_day.deprecated != True)]
        )
        df_stress_metadata_day.append(
            [
                day.date(),
                nb_user_confirmations,
                nb_manual_events,
                nb_user_deleted_events,
                nb_timeline_events,
            ]
        )
    df_stress_interaction = pd.DataFrame(
        data=df_stress_metadata_day,
        columns=[
            "date",
            "nb_confirmed",
            "nb_manual_inserted",
            "nb_manual_deleted",
            "nb_timeline",
        ],
    )
    df_stress_interaction["interaction_rate"] = interaction_rate(df_stress_interaction)
    return df_stress_interaction


def get_activity_interaction_df(df_eventdump) -> pd.DataFrame:
    # analyze the timeline events
    df_activity_events = EventDumpParser.parse_activity_events(df_eventdump)
    if not len(df_activity_events):
        return pd.DataFrame()

    df_activity_events = df_activity_events.set_index("time", drop=True)
    for c in [
        "endTime",
        "predictedEndTime",
        "feedbackTime",
        "sedentaryConfirmed",
        "deprecated",
    ]:
        if c not in df_activity_events.columns:
            df_activity_events[c] = np.NaN

    df_activity_events["prediction"] = (
        df_activity_events["endTime"] == df_activity_events["predictedEndTime"]
    )

    # display(df_activity_events)

    t_start = df_activity_events.index.min()
    t_end = df_activity_events.index.max()
    df_activity_metadata_day = []
    for day in pd.date_range(start=t_start.date(), end=t_end.date(), freq="D"):
        # rate = (manual events + confirmed predicted events) / total_predicted events
        # with a daily mask
        day_mask = df_activity_events.index.date == day.date()

        if not sum(day_mask):
            df_activity_metadata_day.append([day.date(), 0, 0, 0, 0])
            continue

        df_activity_day = df_activity_events[day_mask]

        nb_user_confirmations = len(
            df_activity_day[
                df_activity_day.prediction
                & (
                    df_activity_day["feedbackTime"].notna()
                    | df_activity_day["sedentaryConfirmed"]
                )
                & (df_activity_day.deprecated != True)
            ]
        )

        nb_user_deleted_events = 0
        if "deprecated_by" in df_activity_day.columns:
            nb_user_deleted_events = int(sum(df_activity_day.deprecated_by == "user"))

        nb_manual_events = 0
        nb_manual_events += len(
            df_activity_day[
                # prediction and deprecated can thus be Nan
                (df_activity_day.prediction != True)
                & (df_activity_day.deprecated != False)
            ]
        )
        nb_timeline_events = len(
            df_activity_day[
                df_activity_day.prediction & (df_activity_day.deprecated != True)
            ]
        )
        df_activity_metadata_day.append(
            [
                day.date(),
                nb_user_confirmations,
                nb_manual_events,
                nb_user_deleted_events,
                nb_timeline_events,
            ]
        )
    df_activity_interaction = pd.DataFrame(
        data=df_activity_metadata_day,
        columns=[
            "date",
            "nb_confirmed",
            "nb_manual_inserted",
            "nb_manual_deleted",
            "nb_timeline",
        ],
    )
    df_activity_interaction["interaction_rate"] = interaction_rate(
        df_activity_interaction
    )
    return df_activity_interaction

In [3]:
# ---------------------------- Hyperparameters ---------------------------- #
user = "MBRAIN21-002"
# ------------------------------------------------------------------------- #

# construct the wrist wearable sessions using the temperature data
df_wrist_wearable_sessions = get_wearable_session_df(
    processed_mbrain_path.glob(f"{user}*E4*/tmp*.parquet"), fs_exp=1
)
# use the phone light sensor to get the phone sessions
df_phone_sessions = get_wearable_session_df(
    processed_mbrain_path.glob(f"{user}/light*.parquet"), fs_exp=0.5
)
print(df_phone_sessions.shape)

# load the event dump to get the stress and activity events
df_eventdump = pd.read_csv(mbrain_metadata_path / user / "event_dump.csv")
df_stress_interaction = get_stress_event_interaction_df(df_eventdump)
df_activity_interaction = get_activity_interaction_df(df_eventdump)


dt = df_wrist_wearable_sessions.start[0].replace(
    hour=0, minute=0, second=0, microsecond=0
)
to_same_date = lambda time: datetime.combine(dt.date(), time)

  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

(106, 5)


In [4]:
# ------------------------- create the user figure ------------------------
fig = make_subplots(
    rows=5,
    cols=1,
    shared_xaxes=True,
    subplot_titles=[
        "(I) phone data sessions",
        "(II) user wearable sessions ",
        "(III) daily events",
        "(IV) headaches & medicines ",
        "(V) timeline interactions",
    ],
    vertical_spacing=0.08,
)

# ------------ row 1: add the phone session time
for _, session in df_phone_sessions.iterrows():
    hours, remainder = divmod(session.daily_wear_time.total_seconds(), 3600)
    minutes, seconds = divmod(remainder, 60)

    name = "too short " if session.daily_wear_time < pd.Timedelta(hours=12) else ""
    name += f"phone sessions - {user}"

    fig.add_trace(
        go.Scatter(
            x=[session.date] * 2,
            y=[
                to_same_date(session.start.time()),
                to_same_date(session.end.time()),
            ],
            line=dict(
                color=(
                    "rgba(44, 160, 44, 0.3)"  # cooked asparagus green
                    if session.daily_wear_time > pd.Timedelta(hours=12)
                    else "rgba(255, 127, 14, 0.4)"  # safety orange
                ),
                width=7,
            ),
            mode="markers+lines",
            marker=dict(
                line_color="black",
                line_width=1,
                color="black",
                symbol="line-ew",
                size=6,
                opacity=0.8,
            ),
            name=name,
            showlegend=False,
            hovertext=f"total phone data time = {int(hours)}u{int(minutes)}",
        ),
        row=1,
        col=1,
    )

# row 2: wearable sessions
for _, session in df_wrist_wearable_sessions.iterrows():
    hours, remainder = divmod(session.daily_wear_time.total_seconds(), 3600)
    minutes, seconds = divmod(remainder, 60)

    name = "too short " if session.daily_wear_time < pd.Timedelta(hours=6) else ""
    name += f"wearable sessions - {user}"

    fig.add_trace(
        go.Scatter(
            x=[session.date] * 2,
            y=[
                to_same_date(session.start.time()),
                to_same_date(session.end.time()),
            ],
            line=dict(
                color=(
                    "rgba(44, 160, 44, 0.3)"  # cooked asparagus green
                    if session.daily_wear_time > pd.Timedelta(hours=6)
                    else "rgba(255, 127, 14, 0.4)"  # safety orange
                ),
                width=7,
            ),
            mode="markers+lines",
            marker=dict(
                line_color="black",
                line_width=1,
                color="black",
                symbol="line-ew",
                size=6,
                opacity=0.8,
            ),
            name=name,
            showlegend=False,
            hovertext=f"total daily wear time = {int(hours)}u{int(minutes)}",
        ),
        row=2,
        col=1,
    )

# ---------- row 3: Daily records & headache events
df_daily_record = (
    EventDumpParser.parse_daily_record(df_eventdump).set_index("date").sort_index()
)
df_questionnaire = EventDumpParser.parse_questionnaire_events(df_eventdump)
colors = [
    "#a6cee3",
    "#1f78b4",
    "#b2df8a",
    "#33a02c",
    "#fb9a99",
    "#e31a1c",
    "#fdbf6f",
    "#ff7f00",
    "#cab2d6",
    "#6a3d9a",
    "#ffff99",
    "#b15928",
    "#7fc97f",
    "#beaed4",
    "#fdc086",
    "#ffff99",
    "#386cb0",
    "#f0027f",
    "#bf5b17",
    "#666666",
]
for i, (col, hovertext) in enumerate(
    [
        ("foodIntake.breakfastTime", "breakfast"),
        ("foodIntake.lunchTime", "lunch"),
        ("foodIntake.dinnerTime", "dinner"),
    ]
):
    if col in df_daily_record.columns:
        fig.add_trace(
            go.Scatter(
                x=df_daily_record.index,
                mode="markers",
                name=hovertext,
                y=list(
                    seq(df_daily_record[col]).map(
                        lambda x: (
                            to_same_date(x.time())
                            if not pd.isnull(x) and isinstance(x, datetime)
                            else None
                        )
                    )
                ),
                line=dict(color=colors[i % len(colors)]),
                hovertext=hovertext,
                marker_size=8,
            ),
            row=3,
            col=1,
        )

skipped_legend = False
for col, time in [
    ("foodIntake.skippedBreakfast", "8:30"),
    ("foodIntake.skippedLunch", "12:30"),
    ("foodIntake.skippedDinner", "19:30"),
]:
    if col in df_daily_record.columns:
        h, m = list(map(int, time.split(":")))
        df_skipped = df_daily_record[df_daily_record[col] == True]
        fig.add_trace(
            go.Scatter(
                x=df_skipped.index,
                y=[to_same_date(dt.replace(hour=h, minute=m).time())] * len(df_skipped),
                mode="markers",
                name="skipped food",
                marker_size=8,
                marker=dict(symbol="x", color="red"),
                hovertext=col.split(".")[-1],
                showlegend=not skipped_legend,
            ),
            row=3,
            col=1,
        )
        skipped_legend = True

for q_type, color_str, marker_style, name in [
    ("morning_nl", "rgba(57, 57, 75, 0.55)", "triangle-up", "morning"),
    ("evening_nl", "rgba(57, 57, 75, 0.55)", "triangle-down", "evening"),
    ("stress_nl", "rgba(99, 99, 225, 0.3)", "hash", "stress"),
    ("stress_misprediction_nl", "rgba(99, 99, 225, 0.3)", "hash", ""),
    ("nostress_nl", "rgba(99, 99, 225, 0.3)", "hash", ""),
]:
    df_user_q_type = df_questionnaire[df_questionnaire["payload.name"] == q_type]
    shown = False
    for _, r in df_user_q_type.iterrows():
        fig.add_trace(
            go.Scatter(
                x=[r["time"].date()],
                y=[to_same_date(r["payload.end_time"].time())],
                line=dict(color=color_str),
                mode="markers",
                name=name,
                legend="legend2",
                marker=dict(
                    line_color="black",
                    line_width=1,
                    color=color_str,
                    symbol=marker_style,
                    size=7,
                    opacity=0.5,
                ),
                # name=f"questionnaire - {user}",
                showlegend=name != "" and not shown,
                hovertext=q_type,
            ),
            row=3,
            col=1,
        )
        shown = True

# row 4: Headache and medicine events
df_headache = EventDumpParser.parse_headache_events(df_eventdump)
if "deprecated" in df_headache.columns:
    df_headache = df_headache[df_headache["deprecated"] != True]
df_headache["date"] = pd.to_datetime(df_headache["date"])
df_medicine = EventDumpParser.parse_medicine_events(df_eventdump)
if True:
    shown = False
    for _, r in df_headache.iterrows():
        for d in pd.date_range(
            r.time.replace(hour=0, minute=0, second=0, microsecond=0),
            r.endTime,
            freq="1D",
        ):
            if r.time.date() == d.date():
                start = r.time
            else:
                start = d.replace(hour=0, minute=0, second=0, microsecond=0)
            if r.endTime.date() == d.date():
                end = r.endTime
            else:
                end = d.replace(hour=23, minute=59, second=59, microsecond=0)

            fig.add_trace(
                go.Scatter(
                    x=[d] * 2,
                    y=[to_same_date(start.time()), to_same_date(end.time())],
                    line=dict(color="rgba(255, 0, 0, 0.4)", width=7),  # red
                    mode="markers+lines",
                    name='headache',
                    legend='legend3',
                    marker=dict(
                        line_color="black",
                        line_width=1,
                        color="black",
                        symbol="line-ew",
                        size=6,
                        opacity=0.8,
                    ),
                    showlegend=not shown,
                    hovertext=construct_headache_event_hovertext(r),
                ),
                row=4,
                col=1,
            )
            shown = True
    shown = False
    for _, r in df_medicine.iterrows():
        fig.add_trace(
            go.Scatter(
                x=[r.time.date()] * 2,
                y=[to_same_date(r.time.time())] * 2,
                mode="markers+lines",
                name='medicine',
                legend='legend3',
                marker=dict(
                    line_color="green",
                    line_width=2,
                    color="green",
                    symbol="cross",
                    size=5,
                    opacity=0.8,
                ),
                # legendgroup=user,
                showlegend=not shown,
                hovertext=f"{r['medicine.name']} - {r['medicine.description']}",
            ),
            row=4,
            col=1,
        )
        shown = True


# row 5: Stress and activity interaction visualizations
fig.add_trace(
    go.Scatter(
        x=df_stress_interaction.date,
        y=df_stress_interaction.interaction_rate,
        mode="markers+lines",
        name="stress",
        legend="legend4",
        # same color as the activity interactions
        line=dict(color="lightblue"),
        hovertext=[
            "<br>".join(
                [
                    f"total_events {session.nb_timeline + session.nb_manual_inserted}",
                    f"nbr_manual_inserted: {session.nb_manual_inserted}",
                    f"nbr_manual_deleted: {session.nb_manual_deleted}",
                    f"nbr_confirmed: {session.nb_confirmed}",
                ]
            )
            for _, session in df_stress_interaction.iterrows()
        ],
    ),
    row=5,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df_activity_interaction.date,
        y=df_activity_interaction.interaction_rate,
        mode="markers+lines",
        name="activty",
        legend="legend4",
        # same color as the stress interactions
        line=dict(color="lightgreen"),
        hovertext=[
            "<br>".join(
                [
                    f"total_events {session.nb_timeline + session.nb_manual_inserted}",
                    f"nbr_manual_inserted: {session.nb_manual_inserted}",
                    f"nbr_manual_delete: {session.nb_manual_deleted}",
                    f"nbr_confirmed: {session.nb_confirmed}",
                ]
            )
            for _, session in df_activity_interaction.iterrows()
        ],
    ),
    row=5,
    col=1,
)


# Shade the weekends
dr = pd.date_range(
    df_activity_interaction.date.min(),
    df_activity_interaction.date.max() + pd.Timedelta(days=1),
)
for sat in np.unique(dr[dr.dayofweek == 5]):
    fig.add_vrect(
        x0=pd.Timestamp(sat) - pd.Timedelta(hours=12),
        x1=pd.Timestamp(sat) + pd.Timedelta(hours=36),
        fillcolor="rgba(0, 0, 0, .07)",
        line_width=0,
    )


# ---------------- update the yaxes
for row, col in [(1, 1), (2, 1), (3, 1), (4, 1)]:
    fig.update_yaxes(
        type="date",
        tickformat="%H:%M",
        tickmode="array",
        ticktext=["0:00", "6:00", "12:00", "18:00", "23:59"],
        tickvals=[
            dt.replace(hour=0),
            dt.replace(hour=6),
            dt.replace(hour=12),
            dt.replace(hour=18),
            dt.replace(hour=23, minute=59),
        ],
        range=[dt.replace(hour=0), dt + pd.Timedelta(days=1)],
        autorange='reversed',
        row=row,
        col=col,
    )
fig.update_yaxes(title="interaction rate (%)", row=5, col=1)
fig.update_layout(height=900, title=f"Longitudinal data analysis - {user}", title_x=0.5)
l_props = dict(
    orientation="h",
    yanchor="bottom",
    xanchor="left",
    x=0,
    font_size=15,
    itemsizing="constant",
)
fig.update_layout(
    template="plotly_white",
    margin=dict(l=20, r=20, t=50, b=20),
    legend=dict(**l_props, y=0.57),
    legend2=dict(
        y=0.57,
        title_text='Questionnaire responses:',
        title_font_size=16,
        **dict(
            orientation="h",
            yanchor="bottom",
            xanchor="right",
            x=1,
            font_size=15,
            itemsizing="constant",
        ),
    ),
    legend3=dict(**l_props, y=0.37),
    legend4=dict(**l_props, y=0.14),
)

if USE_PNG or True:  # False:
    fig.show(renderer="png", width=1500, height=950)
    fig.write_html(
        figure_dir / "C1_user_compliance_MBRAIN.html",
        include_plotlyjs="cdn",
        full_html=False,
    )
else:
    fig.show()

ValueError: Failed to start Kaleido subprocess. Error stream:

[0513/110315.451:WARNING:resource_bundle.cc(405)] locale_file_path.empty() for locale 
[0513/110315.514:WARNING:headless_browser_main_parts.cc(83)] Cannot create Pref Service with no user data dir.
[0513/110315.740:ERROR:gpu_init.cc(430)] Passthrough is not supported, GL is swiftshader
